In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import random
from matplotlib.collections import PatchCollection
import os

In [ ]:

# Create output directory for PNG files
output_dir = "pit_cross_section"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Pit 1 geological data with consistent shale and 3.9m total depth
pit_data = pd.DataFrame({
    'Depth from [m]': [0, 1.2, 2.1],
    'Depth to [m]': [1.2, 2.1, 3.9],
    'Material': ['Overburden', 'Shale', 'Coal seam'],
    'Thickness [m]': [1.2, 0.9, 1.8],
    'Description': ['Topsoil/weathered material', 'Shale layer', 'Thick coal seam']
})

# Enhanced realistic colors for coal geology
realistic_colors = {
    'Overburden': '#8B4513',        # Brown soil
    'Shale': '#708090',             # Slate gray for shale
    'Coal seam': '#2F2F2F'          # Dark coal
}

# Surface colors
grass_color = '#228B22'
tree_color = '#654321'
sky_color = '#87CEEB'

def add_coal_texture(ax, x_start, x_end, y_start, y_end, material, base_color):
    """Add realistic texture patterns specific to coal geology"""
    width = x_end - x_start
    height = y_end - y_start

     # Base layer
    rect = patches.Rectangle((x_start, y_start), width, height, 
                           facecolor=base_color, edgecolor='none', alpha=0.9)
    ax.add_patch(rect)
    
    # Add texture based on material type
    if 'overburden' in material.lower():
        # Overburden - mixed soil and weathered rock
        for _ in range(int(width * height * 30)):
            x = random.uniform(x_start, x_end)
            y = random.uniform(y_start, y_end)
            size = random.uniform(0.02, 0.06)
            debris_color = random.choice(['#8B4513', '#A0522D', '#654321', '#D2691E'])
            circle = patches.Circle((x, y), size, color=debris_color, alpha=0.7)
            ax.add_patch(circle)
    
    elif 'shale' in material.lower():
        # Shale - harder bedrock with fractures
        num_layers = max(3, int(height * 15))
        for i in range(num_layers):
            y_layer = y_start + (i * height / num_layers)
            layer_height = height / num_layers
            # Alternating slightly darker/lighter layers
            if i % 2 == 0:
                layer_color = base_color
                alpha = 0.3
            else:
                layer_color = '#5F5F5F'
                alpha = 0.4
            layer_rect = patches.Rectangle((x_start, y_layer), width, layer_height,
                                         facecolor=layer_color, alpha=alpha)
            ax.add_patch(layer_rect)

            # Add rock fractures
        for _ in range(int(width * height * 3)):
            x = random.uniform(x_start, x_end)
            y = random.uniform(y_start, y_end)
            length = random.uniform(0.1, 0.4)
            angle = random.uniform(-np.pi/4, np.pi/4)
            x_end_frac = x + length * np.cos(angle)
            y_end_frac = y + length * np.sin(angle)
            ax.plot([x, x_end_frac], [y, y_end_frac], 'k-', linewidth=0.8, alpha=0.6)
    
    elif 'coal' in material.lower():
        # Coal seam - show cleats and laminations
        # Horizontal laminations (bedding)
        num_laminations = max(10, int(height * 30))
        for i in range(num_laminations):
            y_lam = y_start + (i * height / num_laminations)
            if i % 3 == 0:  # Every third lamination is slightly different
                lam_color = '#1C1C1C'  # Darker coal
                alpha = 0.5
            else:
                lam_color = '#404040'  # Lighter coal
                alpha = 0.3
            lam_rect = patches.Rectangle((x_start, y_lam), width, height/num_laminations,
                                       facecolor=lam_color, alpha=alpha)
            ax.add_patch(lam_rect)
        
        # Vertical cleats (fractures in coal)
        cleat_spacing = 0.2
        for x_cleat in np.arange(x_start + cleat_spacing/2, x_end, cleat_spacing):
            # Add some randomness to cleat position
            x_cleat += random.uniform(-0.05, 0.05)
            ax.plot([x_cleat, x_cleat], [y_start, y_end], 'k-', linewidth=0.8, alpha=0.6)

            # Horizontal cleats
        for y_cleat in np.arange(y_start + 0.1, y_end, 0.15):
            y_cleat += random.uniform(-0.03, 0.03)
            ax.plot([x_start, x_end], [y_cleat, y_cleat], 'k-', linewidth=0.5, alpha=0.4)

def add_pit_details(ax, pit_x_start, pit_x_end, pit_depth):
    """Add realistic coal pit details"""
    # Pit walls - showing exposed geology
    wall_color = '#654321'
    
    # Left wall
    left_wall = patches.Rectangle((pit_x_start - 0.08, -pit_depth), 0.08, pit_depth,
                                facecolor=wall_color, alpha=0.9)
    ax.add_patch(left_wall)
    
    # Right wall  
    right_wall = patches.Rectangle((pit_x_end, -pit_depth), 0.08, pit_depth,
                                 facecolor=wall_color, alpha=0.9)
    ax.add_patch(right_wall)
    
    # Add mining equipment
    # Shovel
    shovel_x = pit_x_start + 0.3
    shovel_y = -0.2
    ax.plot([shovel_x, shovel_x], [shovel_y, shovel_y + 0.6], 'k-', linewidth=3)
    ax.plot([shovel_x - 0.08, shovel_x + 0.08], [shovel_y + 0.6, shovel_y + 0.6], 'k-', linewidth=3)

    # Measuring tape/scale
    tape_x = pit_x_end - 0.4
    for i in range(int(pit_depth) + 1):
        y_mark = -i
        if y_mark >= -pit_depth:
            ax.plot([tape_x, tape_x + 0.05], [y_mark, y_mark], 'r-', linewidth=2)
            ax.text(tape_x + 0.08, y_mark, f'{i}m', fontsize=8, color='red', fontweight='bold')
    
    # Add some coal samples on pit floor
    sample_positions = [pit_x_start + 0.8, pit_x_start + 1.5, pit_x_start + 2.2]
    for sample_x in sample_positions:
        sample_y = -pit_depth + 0.05
        coal_sample = patches.Circle((sample_x, sample_y), 0.05, color='#2F2F2F', alpha=0.9)
        ax.add_patch(coal_sample)

# Create the realistic coal pit cross-section
fig, ax = plt.subplots(figsize=(16, 12))

# Set up the plot area
total_width = 12
pit_width = 3.0
pit_x_start = 4.5
pit_x_end = pit_x_start + pit_width
ground_level = 0
max_depth = 3.9  # Consistent with pit_data total depth

# Add sky
sky_rect = patches.Rectangle((0, ground_level), total_width, 1.5,
                           facecolor=sky_color, alpha=0.3)
ax.add_patch(sky_rect)

# Surface - simple ground line only
ground_line = patches.Rectangle((0, ground_level - 0.05), total_width, 0.05,
                              facecolor='#654321', alpha=0.9)
ax.add_patch(ground_line)

# Add subsurface layers extending beyond pit
for _, layer in pit_data.iterrows():
    depth_from = -layer['Depth from [m]']
    depth_to = -layer['Depth to [m]']
    
    # Full width subsurface layer
    add_coal_texture(ax, 0, total_width, depth_to, depth_from, 
                    layer['Material'], realistic_colors[layer['Material']])
    
    # Add layer labels on the right side
    mid_depth = (depth_from + depth_to) / 2
    thickness = layer['Thickness [m]']
    ax.text(total_width + 0.2, mid_depth, 
            f"{layer['Material']}\n{thickness:.1f}m",
            fontsize=9, verticalalignment='center',
            bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.8))

# Add pit-specific details
add_pit_details(ax, pit_x_start, pit_x_end, max_depth)

# Add geological contacts (boundaries between layers)
for depth in [1.2, 2.1]:
    ax.axhline(y=-depth, color='black', linestyle='-', linewidth=1.5, alpha=0.8)

# Add title and labels
ax.set_title('PIT 1 Cross-Section', 
             fontsize=16, fontweight='bold')
ax.set_xlabel('Distance (m)', fontsize=12)
ax.set_ylabel('Elevation (m)', fontsize=12)

# Set limits and aspect
ax.set_xlim(0, total_width + 3)
ax.set_ylim(-max_depth - 0.5, 1.5)
ax.set_aspect('equal')

# Add grid for reference
ax.grid(True, alpha=0.3)

# Add legend for materials
legend_elements = []
for material, color in realistic_colors.items():
    legend_elements.append(patches.Patch(color=color, label=material))

ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.25, 1))

# Add simplified technical information box
info_text = f"""Total Depth: {max_depth} m
Coal Seam: {pit_data[pit_data['Material'] == 'Coal seam']['Thickness [m]'].sum():.1f} m
Overburden: {pit_data[pit_data['Material'] == 'Overburden']['Thickness [m]'].sum():.1f} m
Shale: {pit_data[pit_data['Material'] == 'Shale']['Thickness [m]'].sum():.1f} m"""

ax.text(0.5, 1.2, info_text, fontsize=10, 
        bbox=dict(boxstyle="round,pad=0.3", facecolor='lightyellow', alpha=0.9))

# Save the figure to the output directory
output_filename = os.path.join(output_dir, "pit1_cross_section.png")
plt.savefig(output_filename, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Cross-section saved as: {output_filename}")

# Show the plot
plt.show()